In [1]:
%load_ext autoreload

import time

import struct
import array
import gc

import numpy as np
import matplotlib.pylab as plt

from data_utils import DataLogger, DataReader

np.set_printoptions(suppress=True, precision=2)

# Load the example data

In [2]:
data = np.load('../data/2021-06-18_11-51-23.npz')
joint_positions = data['joint_positions']

# Store data in mim-data-storage file (.mds)

In [3]:
dl = DataLogger('test.mds')
id_joint_positions = dl.add_field('ctrl.joint_positions', 12)
id_joint_velocities = dl.add_field('ctrl.joint_velocities', 12)
id_slider_positions = dl.add_field('ctrl.slider_positions 😀', 4)

dl.init_file()

joint_positions = data['joint_positions']
joint_velocities = data['joint_velocities']
slider_positions = data['slider_positions']

T = joint_positions.shape[0]


durations = np.zeros(T)
start_overall = time.time()

for it in range(T):
    start = time.time()
    
    dl.begin_timestamp()
    dl.log_array(id_joint_positions, joint_positions[it])
    dl.log_array(id_joint_velocities, joint_velocities[it])
    dl.log_array(id_slider_positions, slider_positions[it])
    dl.end_timestep()
    
#     time.sleep(0.001)
    
    durations[it] = time.time() - start
dl.close_file()

print('Took: %0.3f ms. Max delay: %0.3f ms' % ((time.time() - start_overall) * 1000, np.max(durations) * 1000))

Took: 252.265 ms. Max delay: 0.410 ms


# Read the created mim-data-storage file again

In [4]:
# Reading the file again.
reader = DataReader('test.mds')

# Check if the data we stored and loaded is the same.
assert np.all((reader.data['ctrl.joint_positions'] - joint_positions) == 0.), "Stored and loaded data not the same"

idx: 9999 fields: 3
[('ctrl.joint_positions', 12), ('ctrl.joint_velocities', 12), ('ctrl.slider_positions 😀', 4)]
